In [1]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
from keras.models import load_model
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.training import saver as saver_lib
import tensorflow as tf
import res2net
import os
%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"]='2'
img_size = 600
batch_size = 64
channel = 3
CLASS_NUM = 2
ckpts = './checkpoint2_dir'
K.set_learning_phase(0)
%matplotlib inline


def get_data(file_path):
    image = []
    temp = []
    for root,dirs,files in os.walk(file_path):
        for name in files:
            image.append(os.path.join(root,name))

        for sub_dir in dirs:
            temp.append(os.path.join(root,sub_dir))
            
    labels = []
    for folder in temp:
        n_img = len(os.listdir(folder))
        letter = folder.split('/')[-1]
        labels = np.append(labels,n_img*[letter])
    print(len(labels),len(image))
    temp = np.array([image,labels])
    temp = temp.transpose()#維樹反轉 [2,25000] -> [25000,2]
    np.random.shuffle(temp) #numpy的打亂函數
    print(temp.shape)
    image_list = list(temp[:,0])
    label_list = list(temp[:,1])
    label_list = [int(float(i)) for i in label_list]
    
    return np.array(image_list),np.array(label_list)

def get_train_data(img_list,label_list,shuffle_list,j):
    k = 0
    img = np.zeros([batch_size,img_size,img_size,channel],dtype=np.int32)
    label = []
    for i in range(j*batch_size,(j+1)*batch_size):
        img[k] = cv2.imread(img_list[shuffle_list[i]])
        label.append(label_list[shuffle_list[i]])
        k+=1

    return np.array(img),np.array(label)

def label_convert_onehot(labels):
    n_sample = labels.shape[0]
    n_class = CLASS_NUM
    one_hot_labels = np.zeros((n_sample,n_class))
    one_hot_labels[np.arange(n_sample),labels] = 1
    return np.array(one_hot_labels,dtype = np.uint8)

def show_img(img):
    #plt.savefig("filename.png")
    if(img.shape[-1]==1):
        plt.imshow(img,cmap ='gray')
        plt.show()
    else:
        plt.imshow(img)
        plt.show()

version_path = './'
if(__name__=='__main__'):
    with tf.Graph().as_default():
        test_image_list,test_label_list = get_data('./64_test_img')
        shuffle_test_list = np.arange(test_image_list.shape[0])
        
        trainable = tf.placeholder(tf.bool,name='trainable')
        resnet32 = res2net.Res2net(img_size,channel,None,CLASS_NUM,trainable=False)
        x = resnet32.x
        
        re = resnet32.pre_softmax[:,1]
        one = tf.ones_like(re)
        zero = tf.zeros_like(re)
        result_ = tf.where(re<0.99, x=zero, y=one)
        result = tf.reshape(result_,[-1,1],name='nvidia-tu117cu_bgm3a-fv-rudolph_recombine/outputLayer')
                         
        
        print(result)
        saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            
            if tf.train.latest_checkpoint(ckpts) is not None:
                saver.restore(sess, tf.train.latest_checkpoint(ckpts))
                checkpoint_path = tf.train.latest_checkpoint(ckpts)
                #print(checkpoint_path)
            else:
                assert 'can not find checkpoint folder path!'
                
            
            underkill = 0
            overkill = 0
            total = 0
            for j in range(test_image_list.shape[0]//batch_size):
                batch_x,batch_y = get_train_data(test_image_list,test_label_list,shuffle_test_list,j)
                batch_y = label_convert_onehot(batch_y)
                print(batch_x.shape)
                prediction,softmax = sess.run([result,resnet32.pre_softmax],feed_dict={x:batch_x})
                for k in range(prediction.shape[0]):
                    if(np.argmax(batch_y[k])==0):
                        if(prediction[k]==1):
                            overkill+=1
                    else:
                        if(prediction[k]==0):
                            underkill+=1
                            show_img(batch_x[k])
                            print(softmax[k])
            print('overkill',overkill,'underkill',underkill)
            
            
            snapshot_dir = "snapshot/"
            '''
            checkpoint_path = saver.save(sess, 
                                         snapshot_dir,
                                         global_step=0, 
                                         latest_filename="checkpoint_state")
            print(checkpoint_path)
            '''
            inference_graph = sess.graph

            graph_def_file = "{}{}_int.graphdef".format('./', '123')
            graph_io.write_graph(inference_graph, ".", graph_def_file)

            frozen_model_file = "{}{}.graphdef".format('./', '123')

            freeze_graph.freeze_graph(graph_def_file, "",
                                      False,
                                      checkpoint_path, 
                                      'nvidia-tu117cu_bgm3a-fv-rudolph_recombine/outputLayer', 
                                      "save/restore_all", 
                                      "save/Const:0",
                                      frozen_model_file,
                                      False,
                                      "")



/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.
(64, 64)
(64, 2)
Tensor("nvidia-tu117cu_bgm3a-fv-rudolph_recombine_inputLayer:0", shape=(?, 600, 600, 3), dtype=float32)
Tensor("crop_to_bounding_box/Slice:0", shape=(?, 256, 256, 3), dtype=float32)
Tensor("max_pooling2d/MaxPool:0", shape=(?, 64, 64, 64), dtype=float32)
Tensor("res2net0/res2net01/add:0", shape=(?, 64, 64, 64), dtype=float32)
Tensor("res2net1/res2net16/add:0", shape=(?, 32, 32, 128), dtype=float32)
Tensor("res2net2/res2net234/add:0", shape=(?, 16, 16, 256), dtype=float32)
Tensor("res2net3/res2net31/add:0", shape=(?, 8, 8, 512), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 2), dtype=float32)
Tensor("nvidia-tu117cu_bgm3a-fv-rudolph_recombine/outputLayer:0", shape=(?, 1), dtype=float32)
INFO:tensorflow:Restoring parameters from ./checkpoint2_dir/MyModel-155
(64, 600, 600, 3)
('overkill', 0, 'underkill', 0)
INFO:tensorflow:Restoring parameters from ./checkpoint2_dir/MyModel-155
INFO:tensorflow:Fr

In [4]:
!nvidia-smi -l

Fri Jun 14 02:26:00 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:E0:00.0 Off |                    0 |
| N/A   31C    P0    48W / 350W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:E2:00.0 Off |                    0 |
| N/A   30C    P0    49W / 350W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

+-----------------------------------------------------------------------------+
Fri Jun 14 02:26:30 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:E0:00.0 Off |                    0 |
| N/A   31C    P0    48W / 350W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:E2:00.0 Off |                    0 |
| N/A   30C    P0    49W / 350W |      0MiB / 32480MiB |      0%      Default |
+-------

+-----------------------------------------------------------------------------+
Fri Jun 14 02:27:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:E0:00.0 Off |                    0 |
| N/A   31C    P0    48W / 350W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:E2:00.0 Off |                    0 |
| N/A   30C    P0    49W / 350W |      0MiB / 32480MiB |      0%      Default |
+-------

In [3]:
print('\nwith tf.Session() as sess:\n    sess.run(tf.global_variables_initializer())\n    \n    img,img2 = sess.run([crop_img,resize_img],feed_dict={x:img_})\n    \n    show_img(img[0].astype(int))\n    \n    \n    \n    #saver.save(sess,adam_meta,global_step=0) \n    \n    snapshot_dir = "snapshot/"\n    saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)\n\n    checkpoint_path = saver.save(sess, \n                                 snapshot_dir,\n                                 global_step=0, \n                                 latest_filename="checkpoint_state")\n    inference_graph = sess.graph\n\n    graph_def_file = "{}{}_int.graphdef".format(\'./\', \'123\')\n    graph_io.write_graph(inference_graph, ".", graph_def_file)\n\n    frozen_model_file = "{}{}.graphdef".format(\'./\', \'123\')\n\n    freeze_graph.freeze_graph(graph_def_file, "",\n                              False,\n                              checkpoint_path, \n                              \'crop:0\', \n                              "save/restore_all", \n                              "save/Const:0",\n                              frozen_model_file,\n                              False,\n                              "")\n')


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    img,img2 = sess.run([crop_img,resize_img],feed_dict={x:img_})
    
    show_img(img[0].astype(int))
    
    
    
    #saver.save(sess,adam_meta,global_step=0) 
    
    snapshot_dir = "snapshot/"
    saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)

    checkpoint_path = saver.save(sess, 
                                 snapshot_dir,
                                 global_step=0, 
                                 latest_filename="checkpoint_state")
    inference_graph = sess.graph

    graph_def_file = "{}{}_int.graphdef".format('./', '123')
    graph_io.write_graph(inference_graph, ".", graph_def_file)

    frozen_model_file = "{}{}.graphdef".format('./', '123')

    freeze_graph.freeze_graph(graph_def_file, "",
                              False,
                              checkpoint_path, 
                              'crop:0', 
                              "save/restore_all", 